In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5f37c74a4bd1a2394b4fa5dcbb449244bb183873f839c409c1c2b5d3191175e6de6040979aeb78f7894665a168392bc55c2d58d905f027b397'

In [2]:
import aocd
from aocd.models import Puzzle
day = 20
year = 2024
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [3]:
import heapq
from collections import Counter, defaultdict

In [4]:
test_data = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""
data_test = test_data.splitlines()

In [5]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(141,
 ['#############################################################################################################################################',
  '#...#...###...........#.....#.....###.......#...#.........###...#.....#...#.....#...#...#...#...#...###...#.....#...#...#...#...###...#...###',
  '#.#.#.#.###.#########.#.###.#.###.###.#####.#.#.#.#######.###.#.#.###.#.#.#.###.#.#.#.#.#.#.#.#.#.#.###.#.#.###.#.#.#.#.#.#.#.#.###.#.#.#.###',
  '#.#...#...#...#.......#.#...#...#...#.#.....#.#.#.....#...#...#.#...#...#.#...#.#.#.#.#.#.#.#.#.#.#...#.#.#.#...#.#.#.#.#.#.#.#...#.#...#...#',
  '#.#######.###.#.#######.#.#####.###.#.#.#####.#.#####.#.###.###.###.#####.###.#.#.#.#.#.#.#.#.#.#.###.#.#.#.#.###.#.#.#.#.#.#.###.#.#######.#',
  '#.......#...#.#.....#...#...###...#...#...#...#...#...#...#...#...#.....#.#...#...#.#.#.#.#.#.#...#...#.#...#...#.#.#.#...#.#...#...#.......#',
  '#######.###.#.#####.#.#####.#####.#######.#.#####.#.#####.###.###.#####.#.#.#######.#.#.#.#.#.#####.###.#####

In [6]:
def cx2pos(pos):
    return (int(pos.real), int(pos.imag))

# See better solution for both parts below
def walk(G, start, end, min_safe=0):
    res = []
    stack = [((), 0, 0, start)]
    cnt = 0
    seen = {}
    max_dist = int(1e12)
    
    while len(stack):
        cheats, steps, _, pos = heapq.heappop(stack)
        if steps > max_dist:
            continue
            
        if pos == end:
            if len(cheats) == 0:
                max_dist = steps
            res.append([steps, cheats])
            continue
            
        if (pos, cheats) in seen:
            if seen[(pos, cheats)] <= steps:
                continue
                
        if len(cheats) > 0 and (pos,()) in seen:
            if seen[(pos,())] < steps + min_safe:
                continue
            res.append([max_dist - (seen[(pos,())] - steps), cheats])
            continue
                
        seen[(pos, cheats)] = steps
        
        for delta in [-1, 1, -1j, 1j]:
            if pos+delta not in G:
                continue
            if G[pos+delta] != '#':
                cnt += 1
                heapq.heappush(stack, (cheats, (steps+1), cnt, pos+delta))
            elif len(cheats) < 1:
                cnt += 1
                heapq.heappush(stack, (cheats[:] + (cx2pos(pos+delta),), (steps+1), cnt, pos+delta))
                
            
    return res


In [7]:
d = data_test
d = data
G = {x + 1j*y: c for y, line in enumerate(d) for x, c in enumerate(line)}

start = next(pos for pos in G if G[pos] == 'S')
end = next(pos for pos in G if G[pos] == 'E')

In [8]:
%%time
min_safe = 100
res = walk(G, start, end, min_safe=min_safe)
# res

CPU times: user 290 ms, sys: 0 ns, total: 290 ms
Wall time: 289 ms


In [10]:
cheats_dict = defaultdict(list)
for steps, cht in res:
    cheats_dict[cht].append(steps)

dist_without_cheat = next(steps for steps, cheats in res if len(cheats) == 0)
cnt = Counter([dist_without_cheat-min(steps) for cheats, steps in cheats_dict.items() if min(steps) <= dist_without_cheat-min_safe])
dist_without_cheat#, cnt

9452

In [11]:
sum(cnt.values())

1417

In [12]:
puzzle.answer_a = sum(cnt.values())

In [13]:
# Part 2

In [15]:
def djikstra(G, start, end):
    res = []
    stack = [(0, 0, start)]
    cnt = 0
    seen = {}
    
    while len(stack):
        steps, _, pos = heapq.heappop(stack)
            
        if pos in seen:
            if seen[pos] <= steps:
                continue
                
        seen[pos] = steps
        
        for delta in [-1, 1, -1j, 1j]:
            if pos+delta not in G:
                continue
            if G[pos+delta] != '#':
                cnt += 1
                heapq.heappush(stack, ((steps+1), cnt, pos+delta))
            
    return seen

def mdist(pos0, pos1):
    if isinstance(pos1, complex):
        return int(abs(pos1.real - pos0.real) + abs(pos1.imag - pos0.imag))
    return int(abs(pos1[0] - pos0.real) + abs(pos1[1] - pos0.imag))

def get_cheat_targets(max_cheat_dist=2):
    ans = []
    for y in range(-max_cheat_dist, max_cheat_dist+1):
        for x in range(-max_cheat_dist, max_cheat_dist+1):
            nbr = x+y*1j
            dist = mdist(0, nbr)
            if dist <= max_cheat_dist and dist > 1:
                ans.append((nbr, dist))
    return ans

def find_cheats(fwd, bck, max_cheat_dist=2, min_safe=10):
    # res = defaultdict(list)
    res = {}
    end = next(pos for pos in bck if bck[pos] == 0)
    target = fwd[end]
    nbrs = get_cheat_targets(max_cheat_dist)
    for pos, steps in fwd.items():
        if steps > target - min_safe:
            continue
            
        for nbr, nbr_dist in nbrs:
            npos = pos + nbr
            
            if npos not in bck:
                continue
            
            new_steps = steps + nbr_dist + bck[npos]
            if new_steps <= target - min_safe:
                # res[(pos, npos)].append(target - new_steps)
                res[(pos, npos)] = target - new_steps
    return res
        

In [16]:
d = data_test
d = data
G = {x + 1j*y: c for y, line in enumerate(d) for x, c in enumerate(line)}

start = next(pos for pos in G if G[pos] == 'S')
end = next(pos for pos in G if G[pos] == 'E')

seen = djikstra(G, end, start)
fwd = djikstra(G, start, end)
bck = seen

In [17]:
# Part 1

In [18]:
%%time
cheats = find_cheats(fwd, bck, max_cheat_dist=2, min_safe=100)

# cnt = Counter([min(val) for val in cheats.values()])
cnt = Counter(cheats.values())
res = sum(cnt.values())
res

CPU times: user 16.4 ms, sys: 25.4 ms, total: 41.7 ms
Wall time: 26.9 ms


1417

In [585]:
# Part 2

In [19]:
%%time
cheats = find_cheats(fwd, bck, max_cheat_dist=20, min_safe=100)

CPU times: user 1.75 s, sys: 999 µs, total: 1.75 s
Wall time: 1.75 s


In [20]:
cnt = Counter(cheats.values())
res = sum(cnt.values())
# cnt

In [21]:
res

1014683

In [583]:
puzzle.answer_b = res

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 20! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
